# LFSR synthesis
Let $\mathbb{F}$ be a finite field. Let $(s_0, s_1, \ldots, s_{N-1}) \in \mathbb{F}$ be a finite sequence, find the shortest LFSR $(c_1, c_2, \ldots, c_{L_N}) \in \mathbb{F}$ such that:

$$
s_j + \sum_{i=1}^{L_N} c_is_{j-i} = 0 \; (L_N \leq j \leq N-1)
$$

In [1]:
from sympy import GF, Poly, Symbol
import random

random.seed(0)

SEQLEN = 200
PRIME_MODULUS = 7
gf = GF(PRIME_MODULUS)
x = Symbol("x")

In [2]:
VERBOSE = False
# VERBOSE = True

def get_next_digit(connpoly, internals):
    """Given an LFSR and its internal state return the next digit"""
    if connpoly.degree(x) == 0:
        return gf(0)
    if connpoly.degree(x) > len(internals):
        raise ValueError(f"LFSR has {connpoly.degree()} connections but internal state only has {len(internals)} elements")
    # the constant term is not a connection coefficient
    conn_coeffs = connpoly.all_coeffs()
    output = gf(0)
    for deg in range(1, connpoly.degree(x)+1):
        coeff = conn_coeffs[-1-deg]
        digit = internals[-deg]
        if VERBOSE:
            print(f"c[{deg}] is {coeff}, s[-{deg}] is {digit}")
        output += conn_coeffs[-1-deg] * internals[-deg]
    return -output

def init_bm(symbol, domain):
    cx = Poly(1, symbol, domain=domain)
    n = 0
    bx = Poly(1, symbol, domain=domain)
    dm = domain(1)
    y = 1
    return cx, n, bx, dm, y

def bm(target, symbol, domain):
    cx, n, bx, dm, y = init_bm(symbol, domain)

    while n < len(target):
        dn = target[n] + get_next_digit(cx, target[:n])
        if VERBOSE:
            print(f"d[{n}] is {dn}")
        if dn == 0:
            # use the same LFSR
            y += 1
        elif 2 * cx.degree(x) > len(target):
            # need to adjust LFSR but no length change
            cx = Poly(cx - dn * (dm ** -1) * (x ** y) * bx, x, domain=gf)
            y += 1
        else:
            # length change needed
            tx = cx
            cx = Poly(cx - dn * (dm ** -1) * (x ** y) * bx, x, domain=gf)
            bx = tx
            dm = dn
            y = 1
        n += 1

    return cx

# generate a random sequence
target = [random.randint(0, PRIME_MODULUS-1) for _ in range(10)]
for i, digit in enumerate(target):
    print(f"s[{i}] should be {digit}")
minlfsr = bm(target, x, gf)
display(minlfsr)

for j in range(minlfsr.degree(x), len(target)):
    next_digit = get_next_digit(minlfsr, target[:j])
    print(f"s[{j}] is {next_digit}")

s[0] should be 6
s[1] should be 3
s[2] should be 6
s[3] should be 3
s[4] should be 0
s[5] should be 2
s[6] should be 4
s[7] should be 3
s[8] should be 3
s[9] should be 6


Poly(x**5 - 3*x**4 + x**2 + 2*x + 1, x, modulus=7)

s[5] is 0 mod 7
s[6] is 4 mod 7
s[7] is 0 mod 7
s[8] is 1 mod 7
s[9] is 4 mod 7
